# Script sparcing informe Rettig

## 1. pasar de pdf a html el archivo

Usé este convertidor de pdf a html. Es un wraper de pdfminer pa pasarlo a Html

https://gist.github.com/rguliev/3d886d38daa8ac0be8ddb85d645fb0bc#file-convert_pdf-py

In [1]:
from typing import Container
from io import BytesIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

# definir funcion 
def convert_pdf_to_html(path: str, start_page: int, end_page: int) -> str:
    """Converts a range of pages from a PDF file to HTML format."""
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, "rb")
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page_num, page in enumerate(PDFPage.get_pages(fp), start=1):
        if page_num < start_page:
            continue
        if page_num > end_page:
            break
        interpreter.process_page(page)
    html = retstr.getvalue().decode()
    fp.close()
    device.close()
    retstr.close()
    return html

# definimos file path y paginas correspondientes a casos de RM
pdf_file_path = "informes/Informe-Rettig-tomo1.pdf"
start_page = 139
end_page = 252

# convert  
html_content = convert_pdf_to_html(pdf_file_path, start_page, end_page)

print("html object created")

html object created


In [2]:
# exportamos el html para examinar el archivo 

def export_html_to_file(html_content: str, output_file: str):
    """Exports HTML content to a file."""
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(html_content)

# Define the path for the output HTML file
output_html_file = "data_out/output.html"

# Export HTML content to the output HTML file
export_html_to_file(html_content, output_html_file)
 
print(f"HTML content has been exported to {output_html_file}.")


HTML content has been exported to data_out/output.html.


## 2. maniulacion del HTML para tener una base de datos

### 1. Extracción de lista de nombres tomo I

Usamos beautiful soup para sparcer de html

In [3]:

# sacar todas las cosas en negrita 

from bs4 import BeautifulSoup

def extract_names(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # encontrar las negritas 
    bold_elements = soup.find_all('span', style=lambda value: value and 'font-family: TimesNewRoman,Bold' in value and len(value) > 0)

    # extraer nombres = negritas 
    names = []
    current_name = ""
    for bold_element in bold_elements:
        text = bold_element.get_text().strip()

        # append a la lista if not empty 
        if current_name:
            names.append(current_name)
            current_name = ""

        # incluir todas las palabras en negrita dsps de la primera encontrada
        if bold_element.find_next_sibling():
            # append al nombre
            names.append(text)
        else:
            # guardar el nombre total
            current_name = text

    return names

# output
names = extract_names(html_content)


In [4]:
import re 

#limpiar 

# sacar los nombres que no tienen mayúscula porq no son nombres
# y filtrar las filas vacías
filtered_names = [
    
    name for name in names if any(

    char.isupper() for char in name
    )
    ]

# sacar simbolos q no son letras
cleaned_names = [re.sub(r'[,.-]', '', name) for name in filtered_names]

In [5]:
# pegar nombres rotos 

def concatenate_names(names):
    concatenated_names = []
    prev_name = None

    for name in names:
        if name.isupper():
            if prev_name is not None:
                concatenated_name = prev_name + " " + name
                concatenated_names[-1] = concatenated_name
            else:
                concatenated_names.append(name)
        else:
            concatenated_names.append(name)
            prev_name = name
    
    return concatenated_names

concatenated_names = concatenate_names(cleaned_names)

concatenated_names = [name for name in concatenated_names if name != "Lonquén"]
name_list = concatenated_names[1:]


In [6]:
import pandas as pd

name_list = pd.DataFrame({"names": name_list})
name_list.to_csv('data_out/names_rettig.csv')

name_list

# vemos tabla 

,names
0,Salvador ALLENDE GOSSENS
1,Antonio AGUIRRE VASQUEZ
2,Osvaldo RAMOS \nRIVERA
3,Augusto OLIVARES BECERRA
4,Jaime BARRIOS MEZA
...,...
490,Luis Heriberto CONTRERAS \nESCANILLA
491,Rudy Freddy VIDAL PEREIRA
492,Manuel Tomás ROJAS FUENTES
493,Juan Domingo MARTINEZ ALDANA


### 2. extraccion lista de eventos tomo 1

In [7]:

# funcion extraccion de eventos (no listo)

def extract_events(html_content):
    # parse html 
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # sacar breaks 
    for br in soup.find_all('br'):
        br.unwrap()
    
    # encontrar todas las italicas 
    italic_elements = soup.find_all('span', style=lambda value: value and 'font-family: TimesNewRoman,Italic' in value)
    
    # sacar todo el texto entre italicas como un solo evento 
    events = []
    for italic_element in italic_elements:
        # encontrar primera italic de un parrafo
        parent_div = italic_element.find_parent('div')
        
        if parent_div:
            # init 
            event_text = ""
            
            # iterar sobre los siblings hasta encontrar otra italica 
            next_sibling = parent_div.find_next_sibling()
            while next_sibling:
                
                if 'Italic' in next_sibling.get('style', ''):
                    break
                
                # append parrafo 
                event_text += next_sibling.get_text().strip() + " "
                
                # next child
                next_sibling = next_sibling.find_next_sibling()
            
            # out
            events.append(event_text.strip())
    
    return events

# aplicar sobrre objeto html 
events = extract_events(html_content)

# export 
events_list = pd.DataFrame({"events": events})
events_list.to_csv('data_out/events_rettig.csv')

